In [16]:
from  selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import time
import os
from os.path  import basename
import requests
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys

import datetime, calendar

import re

In [17]:
# Read airports
file_path = os.path.join('/Users/carlosperezricardo/Desktop/TFM/airlines_airports', 'airports_full.csv')

if os.path.exists(file_path):
    airports= pd.read_csv(file_path, index_col=0)
    
else:
    airports_df = pd.read_csv(os.path.join('/Users/carlosperezricardo/Desktop/TFM/airlines_airports', 'airports.csv'))

    airports_code = pd.read_excel(os.path.join('/Users/carlosperezricardo/Desktop/TFM/airlines_airports', 'US_airports.xlsx'), sheet_name='Airports')
    airports_code.columns = airports_code.iloc[0]
    airports_code = airports_code.iloc[1:]
    
    states = pd.read_excel(os.path.join('/Users/carlosperezricardo/Desktop/TFM/airlines_airports', 'US_airports.xlsx'), sheet_name='States')
    states.columns = ['state_name','state_acronym']

    airports = pd.merge(airports_code, states, left_on='City', right_on='state_name', how='left')

    airports[['state_name','state_acronym']] = airports[['state_name','state_acronym']].ffill(axis = 0)
    airports = airports[~airports['FAA'].isnull()]

In [18]:
file_path = os.path.join('/Users/carlosperezricardo/Desktop/TFM/weather', 'weather.csv')

if os.path.exists(file_path):
    weather = pd.read_csv(file_path, index_col=0)

In [19]:
states = ['ATL', 'ORD', 'DFW', 'DEN', 'LAX']

In [20]:
icao_done = weather['ICAO'].value_counts()
icao_done = icao_done[icao_done == 912]
icao_set = list(icao_done.index)
icao_set.append('KBHM')
icao_set = set(icao_set)

In [21]:
def read_table(driver, filas_leyenda, days):
    content = driver.page_source
    soup = BeautifulSoup(content)

    days_month = len(days) + 1

    tabla = soup.find("table",{"class":"days ng-star-inserted"})

    def obtain_row_data (tr, tipo_columna):
        lista_columnas = []

        for td in tr.findAll(tipo_columna):
            lista_columnas.append(td.get_text(strip=True))
            
        return lista_columnas

    def obtenDatosTabla(tabla):
        filas_procesadas = []
        filas = tabla.findAll("tr")

        start = False
        for tr in filas:
            res = obtain_row_data(tr, "td")
            
            if res == ['Max','Avg','Min'] or res == ['Total']:
                start = True
                continue
            if start == True:
                filas_procesadas.append( res )

        return filas_procesadas    

    lista_tabla = obtenDatosTabla(tabla)
    lista_tabla_copy = lista_tabla.copy()

    temperature = pd.DataFrame(lista_tabla[1:days_month])
    lista_tabla_copy = lista_tabla_copy[days_month:]

    dew_point = pd.DataFrame(lista_tabla_copy[1:days_month])
    lista_tabla_copy = lista_tabla_copy[days_month:]

    humidity = pd.DataFrame(lista_tabla_copy[1:days_month])
    lista_tabla_copy = lista_tabla_copy[days_month:]

    wind_speed = pd.DataFrame(lista_tabla_copy[1:days_month])
    lista_tabla_copy = lista_tabla_copy[days_month:]

    pressure = pd.DataFrame(lista_tabla_copy[1:days_month])
    lista_tabla_copy = lista_tabla_copy[days_month:]

    precipitation = pd.DataFrame(lista_tabla_copy[1:days_month])

    add_weather = pd.concat([pd.DataFrame(days), temperature, dew_point, humidity, wind_speed, \
        pressure, precipitation],axis=1)
    add_weather.columns = filas_leyenda
    
    return add_weather

In [22]:
leyenda = ['Temperature','Dew Point','Humidity','Wind Speed','Pressure']
units = ['° F','(° F)','(%)','(mph)','(Hg)','(in)']

In [23]:
filas_leyenda = []
filas_leyenda.append('Day')

for i in leyenda:
    filas_leyenda.append(i+'_Max')
    filas_leyenda.append(i+'_Avg')
    filas_leyenda.append(i+'_Min')

filas_leyenda.append('Precipitation')

In [24]:
main_link = 'https://www.wunderground.com/history'

In [25]:
driver = webdriver.Chrome(ChromeDriverManager().install())



====== WebDriver manager ======
Current google-chrome version is 98.0.4758
Get LATEST driver version for 98.0.4758
Driver [/Users/carlosperezricardo/.wdm/drivers/chromedriver/mac64/98.0.4758.102/chromedriver] found in cache


In [26]:
driver.get(main_link)
time.sleep(3)

In [27]:
# Cookies
try:
    driver.find_element_by_id("truste-consent-button").click()
    time.sleep(1)
except:
    pass

In [28]:
airports = airports[airports.IATA.isin(states)]

In [29]:
airports

,City,FAA,IATA,ICAO,Airport,Role,Enplanements,state_name,state_acronym
57,Los Angeles,LAX,LAX,KLAX,Los Angeles International Airport,P-L,"39,636,042",CALIFORNIA,CA
77,Denver,DEN,DEN,KDEN,Denver International Airport,P-L,"28,267,394",COLORADO,CO
111,Atlanta,ATL,ATL,KATL,Hartsfield–Jackson Atlanta International Airport,P-L,"53,505,795",GEORGIA,GA
138,Chicago,ORD,ORD,KORD,Chicago O'Hare International Airport,P-L,"37,589,899",ILLINOIS,IL
360,Dallas,DFW,DFW,KDFW,Dallas/Fort Worth International Airport,P-L,"31,283,579",TEXAS,TX


stop

In [30]:
stop

NameError: name 'stop' is not defined

In [32]:
weather= pd.DataFrame()

In [33]:
# For airports
# For years 
# For months 
#weather = pd.DataFrame()

for airport in airports['ICAO']:

    # Main link
    driver.get(main_link)
    time.sleep(3)

    driver.find_element_by_name("historySearch").send_keys(airport)
    time.sleep(4)

    # Click view
    #submit-btn button radius ng-star-inserted
    driver.find_element_by_class_name("submit-btn").click()
    time.sleep(2)

    driver.find_element_by_class_name("submit-btn").click()
    time.sleep(1)

    get_url = driver.current_url
    get_url

    # We change url 
    base_url = '/'.join(get_url.replace('daily','monthly').split('/')[:-1])
    base_url

    if airport not in icao_set:

        try:
            for year in [2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021]:
                for month in range(1,13):
                        if year == 2021 and month > 12:
                            pass
                        else:
                            new_url = ''.join([base_url, '/', str(year), '-', str(month)])

                            num_days = calendar.monthrange(year, month)[1]
                            days = [datetime.date(year, month, day).strftime('%Y-%m-%d') for day in range(1, num_days+1)]
                            days

                            driver.get(new_url)
                            time.sleep(4)
                            
                            add_weather = read_table(driver, filas_leyenda, days)
                            add_weather['ICAO'] = airport
                            
                            weather = pd.concat([weather, add_weather], axis=0, ignore_index=True)
                        #print(weather.shape)
                print(year)
                weather.to_csv(os.path.join('/Users/carlosperezricardo/Desktop/TFM/weather', 'weather.csv'))
            print(airport)
        except:
            print('ERROR! ',airport)

2016
2017
2018
2019
2020
2021
KLAX
ERROR!  KDEN
2016
2017
2018
2019
2020
2021
KATL
2016
2017
2018
2019
2020
2021
KORD
ERROR!  KDFW


In [ ]:
weather[weather.ICAO == airport]['Day'].min()

'2019-01-01'

stop

In [ ]:
stop

In [ ]:
year = 2019
month = 1
airport = 'KBHM'

In [ ]:
# Main link
driver.get(main_link)
time.sleep(3)

driver.find_element_by_name("historySearch").send_keys(airport)
time.sleep(4)

# Click view
#submit-btn button radius ng-star-inserted
driver.find_element_by_class_name("submit-btn").click()
time.sleep(2)

driver.find_element_by_class_name("submit-btn").click()
time.sleep(1)

get_url = driver.current_url
get_url

# We change url 
base_url = '/'.join(get_url.replace('daily','monthly').split('/')[:-1])
base_url


'https://www.wunderground.com'

In [ ]:
new_url = ''.join([base_url, '/', str(year), '-', str(month)])

num_days = calendar.monthrange(year, month)[1]
days = [datetime.date(year, month, day).strftime('%Y-%m-%d') for day in range(1, num_days+1)]
days

driver.get(new_url)
time.sleep(4)

add_weather = read_table(driver, filas_leyenda, days)
add_weather['ICAO'] = airport

weather = pd.concat([weather, add_weather], axis=0, ignore_index=True)

AttributeError: 'NoneType' object has no attribute 'findAll'

In [ ]:
content = driver.page_source
soup = BeautifulSoup(content)

days_month = len(days) + 1

tabla = soup.find("table",{"class":"days ng-star-inserted"})

def obtain_row_data (tr, tipo_columna):
    lista_columnas = []

    for td in tr.findAll(tipo_columna):
        lista_columnas.append(td.get_text(strip=True))
        
    return lista_columnas

def obtenDatosTabla(tabla):
    filas_procesadas = []
    filas = tabla.findAll("tr")

    start = False
    for tr in filas:
        res = obtain_row_data(tr, "td")
        
        if res == ['Max','Avg','Min'] or res == ['Total']:
            start = True
            continue
        if start == True:
            filas_procesadas.append( res )

    return filas_procesadas    

lista_tabla = obtenDatosTabla(tabla)
lista_tabla_copy = lista_tabla.copy()

temperature = pd.DataFrame(lista_tabla[1:days_month])
lista_tabla_copy = lista_tabla_copy[days_month:]

dew_point = pd.DataFrame(lista_tabla_copy[1:days_month])
lista_tabla_copy = lista_tabla_copy[days_month:]

humidity = pd.DataFrame(lista_tabla_copy[1:days_month])
lista_tabla_copy = lista_tabla_copy[days_month:]

wind_speed = pd.DataFrame(lista_tabla_copy[1:days_month])
lista_tabla_copy = lista_tabla_copy[days_month:]

pressure = pd.DataFrame(lista_tabla_copy[1:days_month])
lista_tabla_copy = lista_tabla_copy[days_month:]

precipitation = pd.DataFrame(lista_tabla_copy[1:days_month])

add_weather = pd.concat([pd.DataFrame(days), temperature, dew_point, humidity, wind_speed, \
    pressure, precipitation],axis=1)
add_weather.columns = filas_leyenda

In [ ]:
add_weather

,Day,Temperature_Max,Temperature_Avg,Temperature_Min,Dew Point_Max,Dew Point_Avg,Dew Point_Min,Humidity_Max,Humidity_Avg,Humidity_Min,Wind Speed_Max,Wind Speed_Avg,Wind Speed_Min,Pressure_Max,Pressure_Avg,Pressure_Min,Precipitation
0,2019-01-01,53,50.6,48,54,50.4,48,96,85.8,71,16,7.2,0,29.6,29.5,29.5,0.00
1,2019-01-02,55,52.1,50,59,48.4,40,93,76.8,39,5,1.1,0,29.6,29.5,29.4,0.01
2,2019-01-03,62,52.6,43,40,38.1,35,97,73.8,34,14,6.9,0,29.5,29.5,29.4,0.00
3,2019-01-04,60,45.9,40,44,39.5,34,93,60.7,36,21,9.2,5,29.6,29.6,29.6,0.00
4,2019-01-05,67,49.3,36,49,41.8,39,90,72.3,43,16,10.3,6,29.6,29.6,29.6,0.00
5,2019-01-06,68,56.5,42,56,50.9,31,62,41.9,22,12,7.4,0,29.6,29.6,29.5,0.00
6,2019-01-07,70,60.5,52,32,22.6,14,72,52.5,33,7,2.8,0,29.6,29.4,29.3,0.48
7,2019-01-08,53,45.8,37,22,20.3,19,81,52.1,28,18,7.7,0,29.6,29.5,29.4,0.00
8,2019-01-09,47,36.8,29,30,23.3,20,97,78.5,46,14,8.3,0,29.7,29.6,29.6,0.00
9,2019-01-10,54,41.4,27,48,41.7,28,93,84.2,71,12,7.7,5,29.7,29.6,29.6,0.00
